<img src="img/mupiflogo.png" style="float:right;height:80px;">

# Example illustrating indirect API implementation
Indirect API implementation is suitable for closed source codes without any programming interface, so what is typically available is just an executable model and its I/O specification. 

The approach consists of developping API implementation that internally remembers all the inputs (by capturing the API set calls), executing the application itself, and finally parsing the application output to get required quantities, that are later accesible using get API method.

The example below ilustrates the concept on developping a MuPIF API to determine some characteristics (minimum, maximum and average value) of a spatial field. We would like to use externall app (represented by externalapp.py) that on standard input reads a sequence of numbers (one per line), computes their min, max, and average values and prints these characteristics on output. 

We achieve this by sampling the field at evenly spaced positions and sending the obtained values to external app (rectangular domain is assumed).

In [1]:
# local machine only
import sys
sys.path.append('/home/bp/devel/mupif.git')
sys.path.append('.')

import mupif as mp
import models

## API implementation
* Model API is a python class derived from mupif Model class
* API has to definine required metadata, defined by Model metadata schema 

### Concept of indirect API implementation
 <img src="img/indirectAPI1.png" alt="Indirect API concept" style="width: 600px;"/>

In [9]:
import mupif
import numpy as np
import subprocess
class myAPI(mupif.model.Model):
    """
    Simple application that computes the arithmetical average of mapped property using an external code
    """
    def __init__(self, metadata={}):
        if len(metadata) == 0:
            metadata = {
                'Name': 'My application API',
                'ID': 'MyApp 1.0',
                'Description': 'Computes some characteristic of a field',
                'Version_date': '1.0.0, Feb 2019',
                'Geometry': '2D rectangle',
                'Inputs': [
                    {
                        'Name': 'scalar field',
                        'Type': 'mupif.Field',
                        'Required': True,
                        'Type_ID': 'mupif.DataID.PID_Temperature',
                    }
                ],
                'Outputs': [
                    {
                        'Name': 'min',
                        'Type_ID': 'mupif.DataID.PID_Demo_Min',
                        'Type': 'mupif.Property',
                        'Required': False
                    },
                    {
                        'Name': 'max',
                        'Type_ID': 'mupif.DataID.PID_FieldID.FID_Temperature',
                        'Type': 'mupif.Property',
                        'Required': False
                    },
                    {
                        'Name': 'average',
                        'Type_ID': 'mupif.DataID.PID_Demo_Value',
                        'Type': 'mupif.Property',
                        'Required': False
                    }
                ],
                'Solver': {
                    'Software': 'own',
                    'Type': 'none',
                    'Accuracy': 'Medium',
                    'Sensitivity': 'Low',
                    'Complexity': 'Low',
                    'Robustness': 'High',
                    'Estim_time_step_s': 1,
                    'Estim_comp_time_s': 1.e-3,
                    'Estim_execution_cost_EUR': 0.01,
                    'Estim_personnel_cost_EUR': 0.01,
                    'Required_expertise': 'None',
                    'Language': 'Python',
                    'License': 'LGPL',
                    'Creator': 'Borek Patzak',
                    'Version_date': '1.0.0, Feb 2019',
                    'Documentation': 'none',
                },
                'Physics': {
                    'Type': 'Continuum',
                    'Entity': ['Finite volume'],
                    'Equation': ['none'],
                    'Equation_quantities': ['Temperature'],
                    'Relation_description': [],
                    'Relation_formulation': [],
                    'Representation': ''
                }
            }
        super().__init__(metadata=metadata)
        self.min = None
        self.max = None
        self.average = None
        self.field = None
    def get(self, propID, time, objectID=0):
        if (propID == mupif.DataID.PID_Demo_Min):
           return mupif.property.ConstantProperty(value=self.min, propID=mupif.DataID.PID_Demo_Min, valueType=mupif.ValueType.Scalar, unit=mupif.U['K'])
        elif (propID == mupif.DataID.PID_Demo_Max):
           return mupif.property.ConstantProperty(value=self.max, propID=mupif.DataID.PID_Demo_Max, valueType=mupif.ValueType.Scalar, unit=mupif.U['K'])
        elif (propID == mupif.DataID.PID_Demo_Value):
           return mupif.property.ConstantProperty(value=self.average, propID=mupif.DataID.PID_Demo_Value, valueType=mupif.ValueType.Scalar, unit=mupif.U['K'])
        else:
           raise mupif.APIError.APIError ('Unknown property ID')
    def set(self, field, objectID=0):
        if (field.getFieldID() == mupif.DataID.FID_Temperature):
            # remember the mapped value
            self.field = field
        else:
            raise mupif.APIError.APIError ('Unknown field ID')
    def solveStep(self, tstep, stageID=0, runInBackground=False):
        try:
            if (self.field):
                input = ""
                for x in np.linspace(0,5,20):
                    for y in np.linspace(0,1,5):
                        input+=repr(self.field.evaluate((x,y,0)).getValue()[0])
                        input +="\n"
        except Exception as e:
            raise mupif.APIError.ApiError ('Field evaluation failed') from e
            
        try:
            # We create the first subprocess, note that we need stdin=PIPE and stdout=PIPE
            p1 = subprocess.Popen(['python3', 'externalApp.py'], stdin=subprocess.PIPE, stdout=subprocess.PIPE)

            # We immediately run the first subprocess and get the result
            # Note that we encode the data, otherwise we'd get a TypeError
            p1_out = p1.communicate(input=input.encode())
        except Exception as e:
            raise mupif.APIError.APIError ('Subprocess execution failed') from e
        
        tokens=p1_out[0].split()
        self.min = float(tokens[0])
        self.max = float(tokens[1])
        self.average = float(tokens[2])

## Using the API

In [11]:
ts = models.ThermalModel()
m2 = myAPI()

ts.initialize('inputT.in')
bc1 = mp.ConstantProperty(propID=mp.DataID.PID_Temperature, valueType=mp.ValueType.Scalar, value=(50,), unit=mp.U['K'])
bc2 = mp.ConstantProperty(propID=mp.DataID.PID_Temperature, valueType=mp.ValueType.Scalar, value=(0,), unit=mp.U['K'])
ts.set(bc1, 'Cauchy left')
ts.set(bc2, 'Cauchy right')
ts.solveStep(mp.TimeStep(time=0,dt=.1,targetTime=1.,unit=mp.U.s))
# show the field
f=ts.get(mp.DataID.FID_Temperature,time=1.*mp.Q.s)

m2.set (f)
m2.solveStep(mp.TimeStep(time=0,dt=.1,targetTime=1.,unit=mp.U.s))
print("Field min:", m2.get(mp.DataID.PID_Demo_Min,time=1.*mp.Q.s))
print("Field max:", m2.get(mp.DataID.PID_Demo_Max,time=1.*mp.Q.s))
print("Field average:", m2.get(mp.DataID.PID_Demo_Value,time=1.*mp.Q.s))



22:13:39 [1992|MainThread] WARNING:logger.py:211 UnitsWarning: 'W/m/K' contains multiple slashes, which is discouraged by the FITS standard
100%|██████████| 40/40 [00:00<00:00, 14041.86 cells/s]

Field min: 7.142857142857024 K{DataID.PID_Demo_Min,ValueType.Scalar}@None
Field max: 42.85714285714274 K{DataID.PID_Demo_Max,ValueType.Scalar}@None
Field average: 24.99999999999975 K{DataID.PID_Demo_Value,ValueType.Scalar}@None


## Notes on Distributed case 

<img src="img/remoteAPI.png">

* The permanent service to allocate new API instances is needed (see mupif.JobManager and mupif.SimpleJobManager). It provides following functionality:
   * allocation and preallocation of new API instances (when resources available)
   * management of API instances (monitoring and managing individual instances)

* The JobMnager has to register itself to platform Nameserver to allow its discovery.
* Typically, the JobManager and API instances (and API executions) are hosted on the same resource
 
### Remarks to HPC applications
* The execution should be made via HPC scheduling subsystem
* The API solveStep method should prepare the HPC job, schedule the job and waits for its completion
* We will provide (later) generic support for HPC connectivity, but initial implementation can be developped using specific HPC interface in mind. 
* The JobManager and API instances should run on separate resource with access to HPC !

## Recommendations

The real case should perform extensive error checking
* Checking the execution status of external app
* Checking if compulsory parameters were set, etc
* Make sure the application is correctly terminated and resources dealocated 
* ...
